Minimizzare sia per il numero di gruppi da creare sia per il costo delle chiamate api, il cui costo è pari a 1 per ogni gruppo e pari a 1 ogni 100 ip, sempre con il vincolo che al massimo in un gruppo ci possono essere 1000 ip. COPIA DI BACKUP

In [1]:
import pulp
import math

In [2]:
res = {"query": "services.http.response.html_title: \"X Acceleration Codec\" or services.http.response.body: \"X Acceleration Codec\"","field": "location.country","total": 2194,"duration": 1099,"total_omitted": 0,"potential_deviation": 0,"buckets": [{"key": "Netherlands","count": 454},{"key": "Spain","count": 392},{"key": "Germany","count": 267},{"key": "France","count": 246},{"key": "United States","count": 139},{"key": "Brazil","count": 108},{"key": "United Kingdom","count": 59},{"key": "Finland","count": 55},{"key": "Bulgaria","count": 41},{"key": "Italy","count": 39},{"key": "Romania","count": 32},{"key": "Ukraine","count": 32},{"key": "Canada","count": 25},{"key": "Serbia","count": 23},{"key": "Bosnia and Herzegovina","count": 21},{"key": "Thailand","count": 19},{"key": "India","count": 18},{"key": "Turkey","count": 16},{"key": "Singapore","count": 15},{"key": "Colombia","count": 14},{"key": "Pakistan","count": 14},{"key": "Poland","count": 14},{"key": "Lithuania","count": 12},{"key": "Israel","count": 10},{"key": "Portugal","count": 10},{"key": "Iraq","count": 9},{"key": "Russia","count": 8},{"key": "Croatia","count": 7},{"key": "Sweden","count": 7},{"key": "Austria","count": 6},{"key": "Switzerland","count": 6},{"key": "Indonesia","count": 5},{"key": "Belgium","count": 4},{"key": "Costa Rica","count": 4},{"key": "Palestinian Territory","count": 4},{"key": "Albania","count": 3},{"key": "Australia","count": 3},{"key": "Czech Republic","count": 3},{"key": "Estonia","count": 3},{"key": "Macedonia","count": 3},{"key": "Peru","count": 3},{"key": "South Africa","count": 3},{"key": "Tunisia","count": 3},{"key": "Argentina","count": 2},{"key": "Cyprus","count": 2},{"key": "Dominican Republic","count": 2},{"key": "Hungary","count": 2},{"key": "Iceland","count": 2},{"key": "Ireland","count": 2},{"key": "Paraguay","count": 2},{"key": "United Arab Emirates","count": 2},{"key": "Armenia","count": 1},{"key": "Bolivia","count": 1},{"key": "Chile","count": 1},{"key": "Denmark","count": 1},{"key": "Ecuador","count": 1},{"key": "Egypt","count": 1},{"key": "El Salvador","count": 1},{"key": "French Polynesia","count": 1},{"key": "Honduras","count": 1},{"key": "Japan","count": 1},{"key": "Jordan","count": 1},{"key": "Malta","count": 1},{"key": "Montenegro","count": 1},{"key": "Morocco","count": 1},{"key": "Myanmar","count": 1},{"key": "Nepal","count": 1},{"key": "Norway","count": 1},{"key": "Slovakia","count": 1},{"key": "Vietnam","count": 1}]}

In [3]:
data = res['buckets']

In [4]:
len(data)

70

In [5]:
# Maximum capacity per group
capacity = 1000

# Prepare data
countries = [item['key'] for item in data]
counts = [item['count'] for item in data]
num_items = len(countries)
num_bins = num_items  # Upper bound on the number of groups

In [6]:
# Create the problem
prob = pulp.LpProblem("Minimize Total Cost", pulp.LpMinimize)

# Decision variables
x = pulp.LpVariable.dicts('x', ((i, j) for i in range(num_items) for j in range(num_bins)), cat='Binary')
y = pulp.LpVariable.dicts('y', (j for j in range(num_bins)), cat='Binary')
s = pulp.LpVariable.dicts('s', (j for j in range(num_bins)), lowBound=0, cat='Continuous')
c = pulp.LpVariable.dicts('c', (j for j in range(num_bins)), lowBound=0, cat='Integer')

# Objective function: Minimize total cost
prob += pulp.lpSum(y[j] + c[j] for j in range(num_bins))

# Constraints

# 1. Assignment Constraint
for i in range(num_items):
    prob += pulp.lpSum(x[(i, j)] for j in range(num_bins)) == 1

# 2. Group Capacity Constraint
for j in range(num_bins):
    # Total IPs in group j
    prob += s[j] == pulp.lpSum(counts[i] * x[(i, j)] for i in range(num_items))
    # Capacity constraint
    prob += s[j] <= capacity * y[j]

# 3. Variable Relationships
for i in range(num_items):
    for j in range(num_bins):
        prob += x[(i, j)] <= y[j]

# 4. IP Cost Calculation Constraints
for j in range(num_bins):
    # s_j <= 100 * c_j
    prob += s[j] <= 100 * c[j]
    # s_j >= 100 * (c_j - 1) + 1
    prob += s[j] >= 100 * (c[j] - 1) + 1 - (1 - y[j]) * capacity

# 5. Consistency Constraints for Unused Groups
for j in range(num_bins):
    # If y_j == 0, then c_j == 0
    prob += c[j] <= (capacity // 100) * y[j]

# Solve the problem
prob.solve()

/opt/conda/lib/python3.10/site-packages/pulp/pulp.py:1298: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/conda/lib/python3.10/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/da41880179744d1db3f33336f22371f7-pulp.mps -timeMode elapsed -branch -printingOptions all -solution /tmp/da41880179744d1db3f33336f22371f7-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 5325 COLUMNS
At line 35846 RHS
At line 41167 BOUNDS
At line 46208 ENDATA
Problem MODEL has 5320 rows, 5110 columns and 20300 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 24.134 - 0.02 seconds
Cgl0003I 0 fixed, 70 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 5320 rows, 5110 columns (5040 integer (4970 of which binary)) and 20300 elements
Cutoff increment increased from 1e-05 to 0.9999
Cbc0038I Initial state - 33 integers unsatisfied sum - 6.4882
Cbc0038I Pass   1: (0.20 seconds) suminf.    1.2

1

In [7]:
# Solution status
print("Status:", pulp.LpStatus[prob.status])

# Collect used groups
bins = {}
for j in range(num_bins):
    if y[j].varValue > 0.5:
        bin_items = []
        total_count = 0
        for i in range(num_items):
            if x[(i, j)].varValue > 0.5:
                bin_items.append((countries[i], counts[i]))
                total_count += counts[i]
        if total_count > 0:
            group_cost = 1  # Cost per group
            ip_cost = math.ceil(total_count / 100)  # Cost per 100 IPs
            total_group_cost = group_cost + ip_cost
            bins[j] = {
                'items': bin_items,
                'total_count': total_count,
                'group_cost': group_cost,
                'ip_cost': ip_cost,
                'total_group_cost': total_group_cost
            }

# Calculate total cost
total_cost = sum(bin_info['total_group_cost'] for bin_info in bins.values())
total_groups = len(bins)

Status: Optimal


In [8]:
# Output results
print(f"Number of groups used: {total_groups}")
print(f"Total cost (credits spent): {total_cost}")
print("=" * 50)

group_number = 1
for bin_index, bin_info in bins.items():
    print(f"Group {group_number}: Total IPs = {bin_info['total_count']} | Group Cost = {bin_info['group_cost']} credit | IP Cost = {bin_info['ip_cost']} credits | Total Group Cost = {bin_info['total_group_cost']} credits")
    for country, count in bin_info['items']:
        print(f" - {country}: {count}")
    print("-" * 50)
    group_number += 1

Number of groups used: 3
Total cost (credits spent): 25
Group 1: Total IPs = 196 | Group Cost = 1 credit | IP Cost = 2 credits | Total Group Cost = 3 credits
 - Brazil: 108
 - Finland: 55
 - India: 18
 - Colombia: 14
 - Nepal: 1
--------------------------------------------------
Group 2: Total IPs = 998 | Group Cost = 1 credit | IP Cost = 10 credits | Total Group Cost = 11 credits
 - Spain: 392
 - Germany: 267
 - United States: 139
 - United Kingdom: 59
 - Romania: 32
 - Ukraine: 32
 - Thailand: 19
 - Austria: 6
 - Belgium: 4
 - Costa Rica: 4
 - Palestinian Territory: 4
 - Albania: 3
 - Australia: 3
 - Czech Republic: 3
 - Estonia: 3
 - Macedonia: 3
 - Peru: 3
 - South Africa: 3
 - Tunisia: 3
 - Argentina: 2
 - Cyprus: 2
 - Dominican Republic: 2
 - Hungary: 2
 - Iceland: 2
 - Ireland: 2
 - Paraguay: 2
 - United Arab Emirates: 2
--------------------------------------------------
Group 3: Total IPs = 1000 | Group Cost = 1 credit | IP Cost = 10 credits | Total Group Cost = 11 credits
 - N